In [216]:
import sys
sys.version

'3.9.1 (default, Dec 11 2020, 09:29:25) [MSC v.1916 64 bit (AMD64)]'

In [217]:
from pandas_datareader import data as pdr
from datetime import date
import yfinance as yf
yf.pdr_override()
import pandas as pd

In [218]:
# data = pdr.get_data_yahoo('TSLA', start = "2020-01-01", end = "2020-12-31")

In [219]:
# yf.download("SPY AAPL", start="2017-01-01", end="2017-04-30")

In [220]:
symbols = pd.read_csv("../data/nasdaqtraded.txt", delimiter="|")


In [239]:
sample_symbols = symbols.query('`Listing Exchange` == "Q"')['Symbol'].sample(n = 20).str.cat(sep = ' ')

stocks = yf.download(sample_symbols, start="2020-01-01", end="2020-12-31")

yb = stocks['Adj Close'].iloc[0].dropna()
ye = stocks['Adj Close'].iloc[len(stocks)-1].dropna()

stocks_no_na = pd.DataFrame({'YB': yb, 'YE': ye}).dropna()
stocks_no_na['pct_chg'] = stocks_no_na['YE']/stocks_no_na['YB'] - 1
stocks_no_na = stocks_no_na.sort_values('pct_chg', ascending=False)
print(stocks_no_na)
print("Overall gain/loss: {:.2f}%".format(stocks_no_na['pct_chg'].mean()*100))

[*********************100%***********************]  20 of 20 completed

5 Failed downloads:
- CURIW: No data found for this date range, symbol may be delisted
- METXW: No data found for this date range, symbol may be delisted
- BREZW: No data found for this date range, symbol may be delisted
- CMLFW: No data found for this date range, symbol may be delisted
- FATBW: No data found for this date range, symbol may be delisted
              YB          YE    pct_chg
NVAX    4.490000  118.959999  25.494433
POWW    1.120000    3.330000   1.973214
MNKD    1.300000    3.360000   1.584615
CSIQ   22.520000   50.680000   1.250444
VCYT   27.510000   50.990002   0.853508
CPSS    3.400000    4.170000   0.226471
WETF    4.551911    5.330000   0.170937
WSBF   17.261318   18.539267   0.074035
ITIC  144.366104  155.029999   0.073867
ASRV    4.108764    3.180000  -0.226045
TOPS   19.750000    1.200000  -0.939241
Overall gain/loss: 277.60%
